In [39]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv(r"D:\čokoláda\VŠE\Datový projekt\datový projekt\textovka\CleanedLemmaToTf-Idf.txt",engine = 'python')

In [41]:
df = df.iloc[:,[2,12,11]]

In [42]:
df["ConWord"] = np.nan
df["WordVal"] = np.nan

In [43]:
df.head()

,ID,lemma_cleaned,SENTIMENT,ConWord,WordVal
0,933,"remove pass test test , test farce , lecturer ...",Negative,NaN,NaN
1,940,- optional attendance oral exam end,Positive,NaN,NaN
2,200,"Minimum overlap agile development , hard reaction",Negative,NaN,NaN
3,206,"Good people learn language Fact , guest lecture",Positive,NaN,NaN
4,2344,"concept exercise , think exercise base prepare...",Negative,NaN,NaN


In [44]:
df_positive=pd.read_csv(r'D:\čokoláda\VŠE\Datový projekt\datový projekt\textovka\Positive-Idf.txt',engine = 'python')
df_negative=pd.read_csv(r'D:\čokoláda\VŠE\Datový projekt\datový projekt\textovka\Negative-Idf.txt',engine = 'python')

In [45]:
df_positive["Sentiment"] = "Positive"
df_negative["Sentiment"] = "Negative"

In [46]:
df_importance = pd.concat([df_positive,df_negative])

In [47]:
df_importance.shape

(2238, 4)

In [48]:
df_importance.sort_values(by=["importance"],inplace = True,ascending = False)
df_importance.head()

,word,importance,rank,Sentiment
0,exercise,168.192735,1.0,Positive
1,online,102.104435,3.0,Positive
2,practical,101.083419,4.0,Positive
3,practice,91.034144,5.0,Positive
0,lecture,84.810316,1.0,Negative


In [49]:
print(df.dtypes)
df["ConWord"] = df["ConWord"].astype('object') #--> gotta be object otherwise can't store list

ID                 int64
lemma_cleaned     object
SENTIMENT         object
ConWord          float64
WordVal          float64
dtype: object


In [50]:
#for every row creates list of words if the word from df_importance is in
for i in range(len(df)):
    lstwrd = []
    
    for word in list(df_importance.iloc[:,0]):
        try:
            if word in df.iloc[i,1].split(" ",-1):
                lstwrd.append(word)
        except AttributeError:
            lstwrd = []

    if len(lstwrd) > 0:
        df.at[i,"ConWord"] = lstwrd #word
        
    else:
        df.at[i,"ConWord"] = []
        

In [51]:
#explore results
df.tail()

,ID,lemma_cleaned,SENTIMENT,ConWord,WordVal
6156,1820279,problem distance learn .,Positive,"[learn, distance, problem]",NaN
6157,1890545,Basic marketing computer science statistics,Neutral,"[computer, science, marketing]",NaN
6158,1890675,Recording clock,Positive,[clock],NaN
6159,1890707,teamwork,Negative,[teamwork],NaN
6160,1890711,Guests practice,Positive,[practice],NaN


In [52]:
#just run some test
row_value = df.iloc[11,:].copy()
print(len(row_value["ConWord"]))
print(row_value["ConWord"])

4
['subject', 'issue', 'database', 'run']


In [53]:
#create copy of df(just to have some columns)
TestDf = df.iloc[0:2,:].copy()
#clear rows
TestDf = TestDf[0:0]
IterateDf = df.iloc[:,:].copy()

#for every row in iteratedf take that row and store it as row_value 
for row in range(len(IterateDf)):
    row_value = IterateDf.iloc[row,:].copy()

#for every word in row_value column con word
    for word in row_value["ConWord"]:
        #take one of the words and assign it as new value of that col
        row_value["ConWord"] = word
        TestDf = TestDf.append(row_value)
    

In [59]:
#Merge Df & df with importance
ResultDf = pd.merge(TestDf,df_importance, left_on = "ConWord", right_on = "word", how = "left",validate = "m:m")

In [60]:
ResultDf.head()

,ID,lemma_cleaned,SENTIMENT,ConWord,WordVal,word,importance,rank,Sentiment
0,933,"remove pass test test , test farce , lecturer ...",Negative,test,NaN,test,35.921271,9.0,Negative
1,933,"remove pass test test , test farce , lecturer ...",Negative,lecturer,NaN,lecturer,22.076795,46.0,Positive
2,933,"remove pass test test , test farce , lecturer ...",Negative,hour,NaN,hour,14.007415,52.0,Negative
3,933,"remove pass test test , test farce , lecturer ...",Negative,pass,NaN,pass,4.096274,274.0,Positive
4,933,"remove pass test test , test farce , lecturer ...",Negative,remove,NaN,remove,2.018012,470.0,Negative


In [61]:
#Take only relevant cols
ResultDf = ResultDf[["ID","lemma_cleaned","Sentiment","word","importance"]]

In [62]:
#Save result
ResultDf.to_csv(r"D:\čokoláda\VŠE\Datový projekt\datový projekt\textovka\WordsWithImportance.csv")